In [8]:
# plotting of DOS, conductivity and resistivity as magnetic field increases

import numpy as np
from scipy.optimize import brentq
#from scipy.misc import derivative

import matplotlib
import matplotlib.pyplot as plt
# from matplotlib import animation
from matplotlib.animation import FuncAnimation

%matplotlib widget

# https://towardsdatascience.com/matplotlib-animations-in-jupyter-notebook-4422e4f0e389

In [2]:
def DgG(E,B,T,tau,tauq,geff,meff,s):
    wc = q*B/meff # s-1
    chi = 2*np.pi**2*kB*T/(hbar*wc) # dimensionless
    gexp = np.exp(-np.pi*s/(wc*tauq))*np.cos(2*np.pi*s*(E+0.5*geff*muB*B)/(hbar*wc/q)+s*np.pi)*s*chi/(np.sinh(s*chi))
    return gexp

def DgIS(E,B,T,tau,tauq,geff,meff):
    DgE = 0.0
    for s in range(1,Ns):
        DgEsp = DgG(E,B,T,tau,tauq,geff,meff,s)
        DgEsn = DgG(E,B,T,tau,tauq,-geff,meff,s)
        DgE = DgE+DgEsp+DgEsn

    return DgE

def oneplusDgIS(E,B,T,tau,tauq,geff,meff):
    return 1+DgIS(E,B,T,tau,tauq,geff,meff)

def DgEtoEF(EF,B,T,tau,tauq,geff,meff):
    EFnum = 2000
    EFindex = np.arange(EFnum+1)
    EtoEF = (EF*EFindex)/EFnum
    DgInt=oneplusDgIS(EtoEF,B,T,tau,tauq,geff,meff)
    return np.trapz(DgInt,EtoEF)

def ExEf(Ex,EF,B,T,tau,tauq,geff,meff):
    # compare the integral 0 to Ex with EF
    return EF-DgEtoEF(Ex,B,T,tau,tauq,geff,meff)

def diffableN(B,EF,T,tau,tauq,geff,meff):
    N = g0*DgEtoEF(EF,B,T,tau,tauq,geff,meff)
    #print(N)
    return N

def dNdB(EF,B,T,tau,tauq,geff,meff):
    # calculate dN/dB for sigmaXY
    dB = 1.0e-4
    Np = g0*DgEtoEF(EF,B+dB,T,tau,tauq,geff,meff)
    Nm = g0*DgEtoEF(EF,B-dB,T,tau,tauq,geff,meff)
    diff = -(Np-Nm)/(2*dB)
    # print('{:.2f} {:.6e} {:.6e} {:.6e}'.format(B,Np,Nm,diff))
    return diff
'''
def scipydNdB(EF,B,T,tau,tauq,geff,meff):
    diff = derivative(diffableN,B,n=1,dx=1e-6,args=(EF,T,tau,tauq,geff,meff),order=3)
    return diff
'''

'\ndef scipydNdB(EF,B,T,tau,tauq,geff,meff):\n    diff = derivative(diffableN,B,n=1,dx=1e-6,args=(EF,T,tau,tauq,geff,meff),order=3)\n    return diff\n'

In [3]:
hbar = 1.054571817e-34 # Js
me = 9.1093837015e-31 # kg
q = 1.602176634e-19 # C
kB = 1.380649e-23 # J/K
muB = hbar/(2*me) # Bohr magneton in eV

In [4]:
# calculation options

EFmove = True
Ns = 100 # number of harmonics in Poisson summation
Bres = 100 # B points per tesla

In [5]:
# Ge QW parameters

# a = 100e-9 # m
T = 1.6 # K
ns = 9.53e15 # m-2
mu = 10.23 # m2/Vs
B = 7.5 # T
#meff = 0.1*me
meff = 0.080*(1+0.056*(ns/1e15))*me
#print(meff/me)
alpha = 5.68
geff = 3.5

In [6]:
tau = mu*meff/q # s
tauq = tau/alpha # s

kF = np.sqrt(2*np.pi*ns) # m-1

EFk = ((hbar*kF)**2)/(2*meff*q) # eV

g0 = q*meff/(np.pi*hbar**2) # cm-2 eV-1
sigma0 = ns*q*mu # per Ohm per square
rho0 = 1/sigma0 # Ohm per square

Evals = np.linspace(0.000,0.030,3001,endpoint=True) # eV

Bvals = np.linspace(0.01,B,int(B*Bres),endpoint=True)
sigma = np.zeros((len(Bvals),2))
sigmaapprox = np.zeros((len(Bvals),2))
rho = np.zeros((len(Bvals),2))
rhoapprox = np.zeros((len(Bvals),2))

def SdH(Bindex):
    EFvals=np.zeros_like(Bvals)
    B=Bvals[Bindex]
        
    wc = q*B/meff # s-1
    chi = 2*np.pi**2*kB*T/(hbar*wc) # dimensionless

    Eopt = brentq(ExEf,0.5*EFk,1.5*EFk,args=(EFk,B,T,tau,tauq,geff,meff))
    # Eopt=EFk
    # print(EFk,Eopt,DgEtoEF(Eopt,B,T,tau,tauq,geff,meff),ExEf(Eopt,EFk,B,T,tau,tauq,geff,meff))
    EFvals[Bindex] = Eopt
    EF=Eopt
    
    # print('{:.4f}\t{:.4e}'.format(B, g0*DgEtoEF(EF,B,T,tau,tauq,geff,meff)))

    Dg = np.zeros_like(Evals)
    Dg = Dg+DgIS(Evals,B,T,tau,tauq,geff,meff)
    DgEF = DgIS(EF,B,T,tau,tauq,geff,meff)

    sigma0 = ns*q*mu

    neff = ns*(1+DgEF)
    tauR = tau*ns/neff
    # sigmaXX = neff*q*mu/(1+(mu*B)**2)
    # according to the actual papers, n_eff tauR cancels out on top
    sigmaXX = sigma0/(1+(wc*tauR)**2)
    # print(B,dNdB(EF,B,T,tau,tauq,geff,meff),q**2*ns/(meff*wc)*DgEF)
    # dNdBaprx = q*ns/(meff*wc)*DgEF
    #dNdBaprx = ns/B*DgEF
    #sigmaXY = q*dNdBaprx-wc*tauR*sigmaXX    
    #print('{:.2f} {:.6e}'.format(B,dNdBaprx))
    sigmaXY = q*dNdB(EF,B,T,tau,tauq,geff,meff)-wc*tauR*sigmaXX
    # sigmaXY = -wc*tauR*sigmaXX
    # sigmaXY = -ns*q/B + sigmaXX/(wc*tau) # ando:1975:2
    # sigmaXY = sigma0*wc*tauR/(1+(wc*tau)**2)
    sigma[Bindex,:] = sigmaXX, sigmaXY
    sigSquared = 1/(sigmaXX**2+sigmaXY**2)
    rhoXX = sigmaXX*sigSquared
    rhoXY = sigmaXY*sigSquared
    rho[Bindex,:] = rhoXX, rhoXY
    wt2 = (wc*tau)**2
    sigmaXXapprox = sigma0/(1+wt2)*(1+2*wt2/(1+wt2)*DgEF)
    sigmaXYapprox = -sigma0*wc*tau/(1+wt2)*(1-(3*wt2+1)/(wt2*(1+wt2))*DgEF)
    sigmaapprox[Bindex,:] = sigmaXXapprox, sigmaXYapprox
    #sigSquaredapprox = 1/(sigmaXXapprox**2+sigmaXYapprox**2)
    #rhoXXapprox = sigmaXXapprox*sigSquaredapprox
    #rhoXYapprox = sigmaXYapprox*sigSquaredapprox
    rhoXXapprox = 1/sigma0*(1+2*DgEF)
    rhoXYapprox = -wc*tau/sigma0*(1-DgEF/(wc*tau)**2)
    rhoapprox[Bindex,:] = rhoXXapprox, rhoXYapprox
    #print("{:.2f}\t{:.4e}\t{:.4e}\t{:.4f}\t{:.4f}\t{:.4e}\t{:.4e}\t{:.4f}\t{:.4f}".format(B,sigmaXX,-sigmaXY,rhoXX,-rhoXY,sigmaXXapprox,-sigmaXYapprox,rhoXXapprox,-rhoXYapprox))
    #print("{:.2f}\t{:.4e}\t{:.4e}\t{:.4f}\t{:.4f}".format(Bvals[Bindex],sigma[Bindex,0],-sigma[Bindex,1],rho[Bindex,0],-rho[Bindex,1]))



    Bmeas=Bvals<B

In [16]:
plt.figure('DOS and resistivity at B = {:.2f} T'.format(B), figsize=[10,5])

#plt.figure('DOS (B)')
#plt.clf()
ax0=plt.subplot(1,2,1)
ax0.set_xlim(0,30)
ax0.set_xlabel('$E$ [ meV ]')
ax0.set_ylim(0,8)
ax0.set_ylabel('$\\Delta g / g_0$')

ax1=plt.subplot(1,2,2)
ax1.set_xlim(0,7.5)
ax1.set_ylim(0,700)
ax1.set_xlabel('$B$ [ T ]')
ax1.set_ylabel('$\\rho_{xx}$ [ $\\Omega$/$\u25a1$ ]')
ax2=ax1.twinx()
ax2.set_ylim(0,7)
ax2.set_ylabel('$\\rho_{xy}$ [ k$\\Omega$ ]')

anim = FuncAnimation(figure, animate, frames=100, interval=20)
plt.show()


NameError: name 'figure' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
x = np.linspace(0, 2*np.pi, 200)
line, = ax.plot(x, np.sin(x))  # initial line

# update
def update(frame):
    line.set_ydata(np.sin(x + 0.1*frame))  # update y
    return line,  # attention to return the iterable

ani = FuncAnimation(fig, update, frames=100, interval=50, blit=True)

plt.show()